# Message模块

## 简介

`Message`是对大语言模型输入输出信息的封装，可以简化大家和大语言模型的交互。

在`ERNIE Bot Agent`中，主要有如下4类`Message`：

* `HumanMessage`：用户输入给模型的普通信息，比如聊天的问题。
* `SystemMessage`：用户输入给模型的全局信息，比如角色扮演的指令、输出格式设置的指令。
* `AIMessage`：模型返回的信息，比如聊天的回答、触发`Function call`的回答。
* `FunctionMessage`：上一轮模型的输出是带有`Funciton call`的`AIMessage`，则用户需要首先调用`Function`，然后将`Function`的结果输入给大语言模型。

`Message`模块的详细API接口，请参考[文档](https://ernie-bot-agent.readthedocs.io/zh-cn/latest/package/erniebot_agent/message/)。

## 上手使用Message

### 安装

大家参考如下，灵活选用安装ERNIE Bot Agent的方式。

1）源码安装

执行如下命令，使用源码安装ERNIE Bot Agent（要求Python >= 3.8)。



In [ ]:
!git clone https://github.com/PaddlePaddle/ERNIE-Bot-SDK.git
%cd ~/ERNIE-Bot-SDK

# 切换到特定分支，默认是develop分支
# !git checkout -b xxx origin/xxx

# 首先安装Ernie Bot SDK
!pip install ./erniebot/

# 然后安装ERNIE Bot Agent
!pip install ./erniebot-agent 

2）快速安装（暂不支持）

执行如下命令，快速安装最新版本ERNIE Bot Agent（要求Python >= 3.8)。

In [ ]:
!pip install --upgrade erniebot-agent

### 鉴权

大家在使用ERNIE Bot Agent之前，需要完成鉴权步骤：

* 在[AI Studio星河社区](https://aistudio.baidu.com/index)注册并登录账号
* 在个人中心的[访问令牌页面](https://aistudio.baidu.com/index/accessToken)获取用户凭证`Access Token`
* 通过环境变量或者`Python`代码设置`Access Token`

In [ ]:
%env EB_AGENT_ACCESS_TOKEN=xxxxxxx

## 使用Message

首先，导入必要的依赖库。

In [3]:
import os
import json
import asyncio
import erniebot
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

大家在使用`ERNIE Bot SDK`调用文心一言进行多轮对话时，需要按照规范定义每轮对话的信息（如下），稍显复杂。

In [ ]:
erniebot.api_type = "aistudio"
erniebot.access_token = "<access-token-for-aistudio>"
messages = [{
                "role": "user",
                "content": "我在深圳，周末可以去哪里玩"
            }, {
                "role": "assistant",
                "content":"深圳有许多著名的景点，以下是三个推荐景点：1. 深圳世界之窗，2. 深圳欢乐谷，3. 深圳东部华侨城。"
            }, {
                "role": "user",
                "content": "从你推荐的三个景点中，选出最值得去的景点是什么，直接给出景点名字即可"
            }]
response = erniebot.ChatCompletion.create(
    model="ernie-3.5",
    messages=messages
    )

print(response.get_result())

如果基于`ERNIE Bot Agent`调用文心一言，大家使用`Message`模块，可以较好地简化代码。

In [4]:
async def demo():
    model = ERNIEBot(model="ernie-3.5")
    # 使用Message模块
    messages = [HumanMessage("我在深圳，周末可以去哪里玩"),
                AIMessage("深圳有许多著名的景点，以下是三个推荐景点：1. 深圳世界之窗，2. 深圳欢乐谷，3. 深圳东部华侨城。"),
                HumanMessage("从你推荐的三个景点中，选出最值得去的景点是什么，直接给出景点名字即可")]
    ai_message = await model.chat(messages=messages)
    print(ai_message.content)

await demo()

最值得去的景点是深圳欢乐谷。


创建各种`Message`的示例代码如下：

In [5]:
human_message = HumanMessage(content='你好，你是谁？')

system_message = SystemMessage(content='你是一名数学老师，使用浅显易懂的方法来回答问题')

result = {"temperature": 25, "unit": "摄氏度"}
function_message = FunctionMessage(name='get_current_temperature', content=json.dumps(result, ensure_ascii=False))

print(human_message)
print(system_message)
print(function_message)

<role: 'user', content: '你好，你是谁？'>
<role: 'system', content: '你是一名数学老师，使用浅显易懂的方法来回答问题', token_count: 23>
<role: 'function', name: 'get_current_temperature', content: '{"temperature": 25, "unit": "摄氏度"}'>


使用`SystemMessage`的示例代码如下：

In [6]:
async def demo():
    model = ERNIEBot(model="ernie-3.5")
    system_message = SystemMessage(content="你是一名数学老师，尽量使用浅显易懂的方法来解答问题")
    messages = [HumanMessage("勾股定理是什么")]
    ai_message = await model.chat(messages=messages, system=system_message.content)
    print(ai_message.content)

await demo()

勾股定理是一个基本的几何定理，它指出直角三角形的两条直角边的平方和等于斜边的平方。换句话说，如果我们知道直角三角形的两个直角边的长度，我们可以使用勾股定理来找到斜边的长度。这个定理在中国古代被称为勾股定理，也被称为商高定理。在西方，这个定理最早是由古希腊的毕达哥拉斯学派提出的，所以它也被称为毕达哥拉斯定理。

勾股定理是数学中一个非常重要的定理，它可以用代数思想来解决几何问题，也是数形结合的一个重要纽带。这个定理有大约500种证明方法，是数学定理中证明方法最多的定理之一。在中国，周朝时期的商高提出了“勾三股四弦五”的勾股定理的特例。

因此，勾股定理是一个非常强大和有用的工具，它在数学、科学、工程和技术中都有广泛的应用。希望这可以帮到你！
